# Entorno de experimentación `HMMLike`

Entorno dónde se utiliza la minima información en la construcción de las *feature lists*. Esto es bias, la letra actual y la letra anterior. Con esto se simula un HMM pero construido con los CRFs

### Parámetros generales

* Maximo Iteraciones = 50
* K = 3

### Parametros por modelo

* `linearCRF_reg.crfsuite`
    * l1 = 0.1
    * l2 = 0.001
* `HMMLike_baseline.crfsuite`
    * l1 = 0.0
    * l2 = 0.0
* `linearCRF_l1_zero.crfsuite`
    * l1 = 0
    * l2 = 0.001
* `linearCRF_l2_zero.crfsuite`
    * l1 = 0.1
    * l2 = 0



### Importando bibliotecas de python

In [1]:
import os
import sys  
import random
import time
import pycrfsuite
import numpy as np
from sklearn.model_selection import KFold
from utils import (get_corpus, WordsToLetter, accuracy_score, model_trainer,
                   model_tester, write_report, eval_labeled_positions,
                   bio_classification_report)

### Funciones auxiliares    

In [2]:
def get_feature_lists(sent):
    ''' Rules that setting up the feature lists for training

    :param sent: Data as `[[[[[letter, POS, BIO-label],...],words],sents]]`
    :type: list
    :return: list of words with characters as features list:
        [[[[[letterfeatures],POS,BIO-label],letters],words]]
    :rtype: list
    '''
    featurelist = []
    senlen = len(sent)
    # each word in a sentence
    for i in range(senlen):
        word = sent[i]
        wordlen = len(word)
        lettersequence = ''
        # each letter in a word
        for j in range(wordlen):
            letter = word[j][0]
            # gathering previous letters
            lettersequence += letter
            # ignore digits
            if not letter.isdigit():
                features = [
                    'bias',
                    'letterLowercase=' + letter.lower(),
                ]
                if j >= 1:
                    features.append('prevletter=' + lettersequence[j-1:j].lower() + '>')
            featurelist.append(features)
    return featurelist


def get_labels(sent, flag=0):
    labels = []
    for word in sent:
        for letter in word:
            labels.append(letter[2])
    return labels


def sent2features(data):
    return [get_feature_lists(sent) for sent in data]


def sent2labels(data):
    return [get_labels(sent) for sent in data]

### Funciones de Train y Test 

In [3]:
def model_trainer(train_data, hyper):
    """ Entrena un modelo y lo guarda en disco

    Función encargada de entrenar un modelo con base en los hyperparametro y
    lo guarda como un archivo utilizable por `pycrfsuite`

    Parameters
    ----------
    train_data : list
    models_path : str
    hyper : dict
    verbose : bool
    k : int

    Returns
    -------
    train_time : float
        Tiempo de entrenamiento
    compositive_name : str
        Nombre del modelo entrenado
    """
    X_train = sent2features(train_data)
    y_train = sent2labels(train_data)
    
    # Train the model
    trainer = pycrfsuite.Trainer(verbose=True)

    for xseq, yseq in zip(X_train, y_train):
        trainer.append(xseq, yseq)

    # Set training parameters. L-BFGS is default. Using Elastic Net (L1 + L2)
    trainer.set_params({
            'c1': hyper['L1'],  # coefficient for L1 penalty
            'c2': hyper['L2'],  # coefficient for L2 penalty
            'max_iterations': hyper['max-iter']  # early stopping
        })
    # The program saves the trained model to a file:
    start = time.time()
    trainer.train(hyper['path'])
    end = time.time()
    train_time = end - start
    return train_time


def model_tester(test_data, model_path):
    """ Prueba un modelo preentrenado

    Recibe los datos de prueba y realiza las pruebas con el modelo previo

    Parameters
    ----------
    test_data : list
    models_path : str
    model_name : str
    verbose : bool

    Returns
    -------
    y_test : list
        Etiquetas reales
    y_pred : list
        Etiquetas predichas por el modelo
    tagger : Object
        Objeto que etiqueta con base en el modelo
    """
    X_test = sent2features(test_data)
    y_test = sent2labels(test_data)

    # ### Make Predictions
    tagger = pycrfsuite.Tagger()
    # Passing model to tagger
    tagger.open(model_path)  
    # Tagging task using the model
    y_pred = [tagger.tag(xseq) for xseq in X_test]
    # Closing tagger
    tagger.close()
    return y_test, y_pred

## Obteniendo corpus completo

In [4]:
corpus = get_corpus('corpus_otomi_mod', '../corpora/') + \
         get_corpus('corpus_otomi_hard', '../corpora/')
letter_corpus = WordsToLetter(corpus)
dataset = np.array(letter_corpus, dtype=object)

## Parametros base

In [5]:
models_path = 'models/'
env_name = "HMMLike"
max_iter = 50
k = 3
kf = KFold(n_splits=k, shuffle=True)

## Parámetros para `HMMLike_reg.crfsuite`

In [6]:
params = {"L1": 0.1, "L2": 1e-3, "max-iter": max_iter}
variant = "reg"

### Entrenamiento y Tests

In [7]:
%%time
i = 0
full_time = 0
accuracy_set = []
for train_index, test_index in kf.split(dataset):
    i += 1
    train_data, test_data = dataset[train_index], dataset[test_index]
    model_name = f"{env_name}_{variant}_k_{i}.crf"
    params['path'] = os.path.join(models_path, env_name, model_name)
    print("*"*50)
    print(f"Entrenando nuevo modelo '{model_name}' | K = {i}")
    print(f"len train: {len(train_data)} len test: {len(test_data)}")
    print("*"*50)
    train_time = model_trainer(train_data, params)
    full_time += train_time
    print("*"*50)
    print(f"Tiempo de entrenamiento: {train_time}[s] | {train_time / 60}[m]")
    print("Test del modelo")
    y_test, y_pred = model_tester(test_data, params['path'])
    accuracy_set.append(accuracy_score(y_test, y_pred))
    print(f"Partial accuracy: {accuracy_set[i - 1]}\n")
    # Reports
    eval_labeled_positions(y_test, y_pred)
    print(bio_classification_report(y_test, y_pred))
print("\n\nAccuracy Set -->", accuracy_set)
train_time_format = str(round(full_time / 60, 2)) + "[m]"
print(f"\nTime>> {train_time_format}")
train_size = len(train_data)
test_size = len(test_data)
params['k-folds'] = k
write_report(model_name, train_size, test_size, accuracy_set, train_time_format,
             params)

**************************************************
Entrenando nuevo modelo 'HMMLike_reg_k_1.crf' | K = 1
len train: 1179 len test: 590
**************************************************
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 1890
Seconds required: 0.013

L-BFGS optimization
c1: 0.100000
c2: 0.001000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 127404.408866
Feature norm: 1.000000
Error norm: 17965.665587
Active features: 1870
Line search trials: 1
Line search step: 0.000055
Seconds required for this iteration: 3.524

***** Iteration #2 *****
Loss: 91837.758116
Feature norm: 5.672465
Error norm: 23376.265302
Active features: 1831
Line search trials: 6
Line search step: 0.031250
Seconds required for this iteration: 10.986



/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

     B-1.cnt       1.00      1.00      1.00        18
     I-1.cnt       0.97      1.00      0.99        36
     B-1.cpl       0.84      0.99      0.90        72
     I-1.cpl       0.84      0.99      0.90        72
 B-1.cpl.irr       0.80      0.80      0.80         5
 I-1.cpl.irr       0.80      0.80      0.80         5
     B-1.enf       0.62      0.73      0.67        11
     I-1.enf       0.62      0.73      0.67        11
     B-1.icp       0.88      1.00      0.94        75
     I-1.icp       0.88      1.00      0.94        75
 B-1.icp.irr       1.00      0.71      0.83         7
 I-1.icp.irr       1.00      0.71      0.83        14
     B-1.irr       0.00      0.00      0.00         1
     I-1.irr       0.00      0.00      0.00         2
     B-1.obj       0.82      0.74      0.78        43
     I-1.obj       0.82      0.74      0.78        43
     B-1.pot       0.80      0.94      0.86        71
     I-1.pot       0.80    

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


***** Iteration #2 *****
Loss: 99979.249639
Feature norm: 6.130083
Error norm: 23933.871904
Active features: 1919
Line search trials: 6
Line search step: 0.031250
Seconds required for this iteration: 13.285

***** Iteration #3 *****
Loss: 74859.466502
Feature norm: 4.486744
Error norm: 13181.504634
Active features: 1856
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 2.198

***** Iteration #4 *****
Loss: 66557.958565
Feature norm: 5.670793
Error norm: 6385.531293
Active features: 1874
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 2.175

***** Iteration #5 *****
Loss: 64292.447775
Feature norm: 6.356777
Error norm: 5636.672706
Active features: 1860
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 2.240

***** Iteration #6 *****
Loss: 63206.839180
Feature norm: 6.844862
Error norm: 7389.210236
Active features: 1886
Line search trials: 1
Line search step: 1.000000
Seconds req

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Parámetros para `HMMLike_l1_zero.crfsuite`

In [8]:
params = {"L1": 0.0, "L2": 1e-3, "max-iter": max_iter}
variant = "l1_zero"

### Entrenamiento y Tests

In [9]:
%%time
i = 0
full_time = 0
accuracy_set = []
for train_index, test_index in kf.split(dataset):
    i += 1
    train_data, test_data = dataset[train_index], dataset[test_index]
    model_name = f"{env_name}_{variant}_k_{i}.crf"
    params['path'] = os.path.join(models_path, env_name, model_name)
    print("*"*50)
    print(f"Entrenando nuevo modelo '{model_name}' | K = {i}")
    print(f"len train: {len(train_data)} len test: {len(test_data)}")
    print("*"*50)
    train_time = model_trainer(train_data, params)
    full_time += train_time
    print("*"*50)
    print(f"Tiempo de entrenamiento: {train_time}[s] | {train_time / 60}[m]")
    print("Test del modelo")
    y_test, y_pred = model_tester(test_data, params['path'])
    accuracy_set.append(accuracy_score(y_test, y_pred))
    print(f"Partial accuracy: {accuracy_set[i - 1]}\n")
    # Reports
    eval_labeled_positions(y_test, y_pred)
    print(bio_classification_report(y_test, y_pred))
print("\n\nAccuracy Set -->", accuracy_set)
train_time_format = str(round(full_time / 60, 2)) + "[m]"
print(f"\nTime>> {train_time_format}")
train_size = len(train_data)
test_size = len(test_data)
params['k-folds'] = k
write_report(model_name, train_size, test_size, accuracy_set, train_time_format,
             params)

**************************************************
Entrenando nuevo modelo 'HMMLike_l1_zero_k_1.crf' | K = 1
len train: 1179 len test: 590
**************************************************
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 1988
Seconds required: 0.014

L-BFGS optimization
c1: 0.000000
c2: 0.001000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 84949.789765
Feature norm: 5.000000
Error norm: 19760.455542
Active features: 1988
Line search trials: 2
Line search step: 0.000271
Seconds required for this iteration: 6.894

***** Iteration #2 *****
Loss: 76434.332177
Feature norm: 4.426436
Error norm: 7632.505925
Active features: 1988
Line search trials: 2
Line search step: 0.434758
Seconds required for this iteration: 4.812


/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



L-BFGS optimization
c1: 0.000000
c2: 0.001000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 84029.351678
Feature norm: 5.000000
Error norm: 19588.917742
Active features: 1941
Line search trials: 2
Line search step: 0.000281
Seconds required for this iteration: 5.610

***** Iteration #2 *****
Loss: 75418.502300
Feature norm: 4.401068
Error norm: 7319.427294
Active features: 1941
Line search trials: 2
Line search step: 0.434717
Seconds required for this iteration: 3.727

***** Iteration #3 *****
Loss: 70388.790886
Feature norm: 4.937088
Error norm: 5872.931318
Active features: 1941
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.865

***** Iteration #4 *****
Loss: 64958.690014
Feature norm: 6.002119
Error norm: 3520.104780
Active features: 1941
Line search trials: 2
Line search step: 0.355470
Seconds required for this iteration: 3.

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 1881
Seconds required: 0.008

L-BFGS optimization
c1: 0.000000
c2: 0.001000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 85868.174796
Feature norm: 5.000000
Error norm: 20617.232924
Active features: 1881
Line search trials: 2
Line search step: 0.000271
Seconds required for this iteration: 5.018

***** Iteration #2 *****
Loss: 85108.063099
Feature norm: 3.713908
Error norm: 15857.996169
Active features: 1881
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.664

***** Iteration #3 *****
Loss: 71097.748675
Feature norm: 4.854065
Error norm: 9242.575331
Active features: 1881
Line search trials: 1
Line search step: 1.000000
Seconds required for this 

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Parámetros para `HMMLike_l2_zero.crfsuite`

In [10]:
params = {"L1": 0.1, "L2": 0.0, "max-iter": max_iter}
variant = "l2_zero"

### Entrenamiento y Tests

In [11]:
%%time
i = 0
full_time = 0
accuracy_set = []
for train_index, test_index in kf.split(dataset):
    i += 1
    train_data, test_data = dataset[train_index], dataset[test_index]
    model_name = f"{env_name}_{variant}_k_{i}.crf"
    params['path'] = os.path.join(models_path, env_name, model_name)
    print("*"*50)
    print(f"Entrenando nuevo modelo '{model_name}' | K = {i}")
    print(f"len train: {len(train_data)} len test: {len(test_data)}")
    print("*"*50)
    train_time = model_trainer(train_data, params)
    full_time += train_time
    print("*"*50)
    print(f"Tiempo de entrenamiento: {train_time}[s] | {train_time / 60}[m]")
    print("Test del modelo")
    y_test, y_pred = model_tester(test_data, params['path'])
    accuracy_set.append(accuracy_score(y_test, y_pred))
    print(f"Partial accuracy: {accuracy_set[i - 1]}\n")
    # Reports
    eval_labeled_positions(y_test, y_pred)
    print(bio_classification_report(y_test, y_pred))
print("\n\nAccuracy Set -->", accuracy_set)
train_time_format = str(round(full_time / 60, 2)) + "[m]"
print(f"\nTime>> {train_time_format}")
train_size = len(train_data)
test_size = len(test_data)
params['k-folds'] = k
write_report(model_name, train_size, test_size, accuracy_set, train_time_format,
             params)

**************************************************
Entrenando nuevo modelo 'HMMLike_l2_zero_k_1.crf' | K = 1
len train: 1179 len test: 590
**************************************************
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 1911
Seconds required: 0.015

L-BFGS optimization
c1: 0.100000
c2: 0.000000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 127423.875123
Feature norm: 1.000000
Error norm: 17932.362539
Active features: 1898
Line search trials: 1
Line search step: 0.000055
Seconds required for this iteration: 3.627

***** Iteration #2 *****
Loss: 94412.222773
Feature norm: 5.888778
Error norm: 23422.599908
Active features: 1857
Line search trials: 6
Line search step: 0.031250
Seconds required for this iteration: 11.2

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 1964
Seconds required: 0.009

L-BFGS optimization
c1: 0.100000
c2: 0.000000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 129604.176838
Feature norm: 1.000000
Error norm: 17997.926564
Active features: 1948
Line search trials: 1
Line search step: 0.000055
Seconds required for this iteration: 4.316

***** Iteration #2 *****
Loss: 98672.394742
Feature norm: 6.145653
Error norm: 23866.300494
Active features: 1924
Line search trials: 6
Line search step: 0.031250
Seconds required for this iteration: 12.794

***** Iteration #3 *****
Loss: 74008.980288
Feature norm: 4.530156
Error norm: 13043.445017
Active features: 1864
Line search trials: 1
Line search step: 1.000000
Seconds required for th

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



L-BFGS optimization
c1: 0.100000
c2: 0.000000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 131368.450588
Feature norm: 1.000000
Error norm: 18298.861170
Active features: 1925
Line search trials: 1
Line search step: 0.000054
Seconds required for this iteration: 4.013

***** Iteration #2 *****
Loss: 98331.814548
Feature norm: 5.909255
Error norm: 24201.747279
Active features: 1875
Line search trials: 6
Line search step: 0.031250
Seconds required for this iteration: 11.975

***** Iteration #3 *****
Loss: 77466.022783
Feature norm: 4.343888
Error norm: 14034.084886
Active features: 1872
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 2.078

***** Iteration #4 *****
Loss: 67938.307111
Feature norm: 5.488134
Error norm: 5071.328436
Active features: 1861
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Parámetros para `baseline.crfsuite`

In [12]:
params = {"L1": 0.0, "L2": 0.0, "max-iter": max_iter}
variant = "baseline"

### Entrenamiento y Tests

In [13]:
%%time
i = 0
full_time = 0
accuracy_set = []
for train_index, test_index in kf.split(dataset):
    i += 1
    train_data, test_data = dataset[train_index], dataset[test_index]
    model_name = f"{env_name}_{variant}_k_{i}.crf"
    params['path'] = os.path.join(models_path, env_name, model_name)
    print("*"*50)
    print(f"Entrenando nuevo modelo '{model_name}' | K = {i}")
    print(f"len train: {len(train_data)} len test: {len(test_data)}")
    print("*"*50)
    train_time = model_trainer(train_data, params)
    full_time += train_time
    print("*"*50)
    print(f"Tiempo de entrenamiento: {train_time}[s] | {train_time / 60}[m]")
    print("Test del modelo")
    y_test, y_pred = model_tester(test_data, params['path'])
    accuracy_set.append(accuracy_score(y_test, y_pred))
    print(f"Partial accuracy: {accuracy_set[i - 1]}\n")
    # Reports
    eval_labeled_positions(y_test, y_pred)
    print(bio_classification_report(y_test, y_pred))
print("\n\nAccuracy Set -->", accuracy_set)
train_time_format = str(round(full_time / 60, 2)) + "[m]"
print(f"\nTime>> {train_time_format}")
train_size = len(train_data)
test_size = len(test_data)
params['k-folds'] = k
write_report(model_name, train_size, test_size, accuracy_set, train_time_format,
             params)

**************************************************
Entrenando nuevo modelo 'HMMLike_baseline_k_1.crf' | K = 1
len train: 1179 len test: 590
**************************************************
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 1886
Seconds required: 0.013

L-BFGS optimization
c1: 0.000000
c2: 0.000000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 84534.247375
Feature norm: 5.000000
Error norm: 20138.686320
Active features: 1886
Line search trials: 2
Line search step: 0.000274
Seconds required for this iteration: 5.290

***** Iteration #2 *****
Loss: 84049.859816
Feature norm: 3.745593
Error norm: 15704.407962
Active features: 1886
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.75

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 1996
Seconds required: 0.010

L-BFGS optimization
c1: 0.000000
c2: 0.000000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 85212.623145
Feature norm: 5.000000
Error norm: 19799.331966
Active features: 1996
Line search trials: 2
Line search step: 0.000273
Seconds required for this iteration: 6.232

***** Iteration #2 *****
Loss: 76607.456086
Feature norm: 4.413827
Error norm: 7514.284102
Active features: 1996
Line search trials: 2
Line search step: 0.431647
Seconds required for this iteration: 4.158

***** Iteration #3 *****
Loss: 71371.583566
Feature norm: 4.953939
Error norm: 6026.498350
Active features: 1996
Line search trials: 1
Line search step: 1.000000
Seconds required for this i

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



L-BFGS optimization
c1: 0.000000
c2: 0.000000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 85104.149569
Feature norm: 5.000000
Error norm: 20002.105414
Active features: 1935
Line search trials: 2
Line search step: 0.000276
Seconds required for this iteration: 5.871

***** Iteration #2 *****
Loss: 76217.819522
Feature norm: 4.396731
Error norm: 7476.715633
Active features: 1935
Line search trials: 2
Line search step: 0.441140
Seconds required for this iteration: 3.934

***** Iteration #3 *****
Loss: 71047.054365
Feature norm: 4.945953
Error norm: 5912.344112
Active features: 1935
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.968

***** Iteration #4 *****
Loss: 65725.442893
Feature norm: 6.009913
Error norm: 3665.871805
Active features: 1935
Line search trials: 2
Line search step: 0.375239
Seconds required for this iteration: 3.

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
